In [ ]:
!unzip /content/sample_data/processed.zip -d /content/sample_data/

: 

In [ ]:
# Install required packages (if not already installed in Colab)
!pip install numpy pandas scikit-learn matplotlib

# Import libraries
import os
import numpy as np
import pandas as pd
from typing import List, Dict
from dataclasses import dataclass
import math
import argparse
from google.colab import files  # For file upload in Colab
import io
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from google.colab import files
from scipy.stats import pointbiserialr

print("Dependencies and imports set up.")

In [ ]:
# Set the base directory containing the abnormal and normal subfolders
base_directory = '/content/sample_data/processed/10'

# Define subdirectories for abnormal and normal tracks
abnormal_dir = os.path.join(base_directory, 'abnormal')
normal_dir = os.path.join(base_directory, 'normal')

# Step 1: Load CSV files from both directories
def load_trajectories(abnormal_dir, normal_dir):
    tracks = {}

    # Load abnormal tracks
    if os.path.exists(abnormal_dir):
        abnormal_files = [f for f in os.listdir(abnormal_dir) if f.endswith('.csv')]
        for csv_file in abnormal_files:
            file_path = os.path.join(abnormal_dir, csv_file)
            try:
                df = pd.read_csv(file_path)[['frameNo', 'left', 'top', 'w', 'h']]
                if not df.empty:
                    # Extract track ID (first number before '_', or full filename if no '_')
                    track_id = csv_file.split('_')[0] if '_' in csv_file else csv_file.replace('.csv', '')
                    tracks[track_id] = {'df': df, 'label': 0}  # 0 for abnormal
                    print(f"Loaded {csv_file} from abnormal folder with {len(df)} rows for track ID {track_id}.")
                else:
                    print(f"Warning: {csv_file} is empty and will be skipped.")
            except Exception as e:
                print(f"Error loading {csv_file} from abnormal folder: {e}")
    else:
        print(f"Warning: Abnormal directory {abnormal_dir} not found.")

    # Load normal tracks
    if os.path.exists(normal_dir):
        normal_files = [f for f in os.listdir(normal_dir) if f.endswith('.csv')]
        for csv_file in normal_files:
            file_path = os.path.join(normal_dir, csv_file)
            try:
                df = pd.read_csv(file_path)[['frameNo', 'left', 'top', 'w', 'h']]
                if not df.empty:
                    # Extract track ID (first number before '_', or full filename if no '_')
                    track_id = csv_file.split('_')[0] if '_' in csv_file else csv_file.replace('.csv', '')
                    tracks[track_id] = {'df': df, 'label': 1}  # 1 for normal
                    print(f"Loaded {csv_file} from normal folder with {len(df)} rows for track ID {track_id}.")
                else:
                    print(f"Warning: {csv_file} is empty and will be skipped.")
            except Exception as e:
                print(f"Error loading {csv_file} from normal folder: {e}")
    else:
        print(f"Warning: Normal directory {normal_dir} not found.")

    return tracks

# Step 2: Compute centers
def compute_centers(tracks):
    for track_id, data in tracks.items():
        df = data['df']
        df['center_x'] = df['left'] + df['w'] / 2
        df['center_y'] = df['top'] + df['h'] / 2

# Step 3: Plot all trajectories in one image
def plot_all_trajectories(tracks):
    plt.figure(figsize=(12, 8))
    colors = {'0': 'red', '1': 'green'}  # Red for abnormal, Green for normal

    for track_id, data in tracks.items():
        df = data['df']
        label = str(data['label'])
        color = colors[label]

        # Plot the trajectory without labels
        plt.scatter(df['center_x'], df['center_y'], c=color, alpha=0.5)
        plt.plot(df['center_x'], df['center_y'], c=color, alpha=0.3)

    # Remove labels, title, and legend
    plt.title('')  # Empty title
    plt.xlabel('')  # Empty x-axis label
    plt.ylabel('')  # Empty y-axis label
    plt.grid(False)  # No grid
    plt.xticks([])  # Remove x-axis ticks
    plt.yticks([])  # Remove y-axis ticks

    # Save the plot as an image
    output_path = os.path.join(base_directory, 'all_trajectories.png')
    plt.savefig(output_path, bbox_inches='tight')
    print(f"Saved plot to {output_path}")

    # Show the plot
    plt.show()

# Main execution
if __name__ == "__main__":
    # Load trajectories from both folders
    tracks = load_trajectories(abnormal_dir, normal_dir)
    if not tracks:
        print("No valid trajectories loaded. Exiting.")
        exit()

    # Compute centers
    compute_centers(tracks)

    # Plot all trajectories in one image
    plot_all_trajectories(tracks)

In [ ]:
# Define a data class for a vehicle's track point
@dataclass
class TrackPoint:
    frame_id: int  # Frame number from CSV (continuous)
    x: float       # 'left' coordinate of bounding box
    y: float       # 'top' coordinate of bounding box
    zone_id: int   # Assigned zone ID

VehicleTrack = List[TrackPoint]

# Define the TrafficAnalyzer class for feature extraction
class TrafficAnalyzer:
    def __init__(self, fps: float, speed_limit: float, zone_definitions: Dict,
                 violation_dict: Dict[str, List[str]], gsd: float):
        """Initialize with project-specific parameters."""
        self.fps = fps
        self.speed_limit = speed_limit
        self.zone_definitions = zone_definitions
        self.violation_dict = violation_dict
        self.gsd = gsd

    def calculate_speed(self, track: VehicleTrack) -> List[float]:
        """Calculate speed using frame differences as time intervals."""
        speeds = []
        for i in range(1, len(track)):
            dx = (track[i].x - track[i-1].x) * self.gsd
            dy = (track[i].y - track[i-1].y) * self.gsd
            distance = math.sqrt(dx**2 + dy**2)
            time = (track[i].frame_id - track[i-1].frame_id) / self.fps
            speed_mps = distance / time if time > 0 else 0
            speed_kmph = speed_mps * 3.6
            speeds.append(speed_kmph)
        return speeds

    def calculate_turn_angle(self, track: VehicleTrack) -> List[float]:
        """Calculate turn angles using three consecutive points."""
        angles = []
        for i in range(2, len(track)):
            v1 = (track[i-1].x - track[i-2].x, track[i-1].y - track[i-2].y)
            v2 = (track[i].x - track[i-1].x, track[i].y - track[i-1].y)
            dot_product = v1[0] * v2[0] + v1[1] * v2[1]
            mag_v1 = math.sqrt(v1[0]**2 + v1[1]**2)
            mag_v2 = math.sqrt(v2[0]**2 + v2[1]**2)
            if mag_v1 * mag_v2 == 0:
                angles.append(0)
                continue
            cos_theta = dot_product / (mag_v1 * mag_v2)
            cos_theta = min(1.0, max(-1.0, cos_theta))
            angle = math.degrees(math.acos(cos_theta))
            angles.append(angle)
        return angles

    def get_zone_sequence(self, track: VehicleTrack) -> List[int]:
        """Get sequence of zones traversed."""
        return [point.zone_id for point in track]

    def detect_abnormalities(self, track: VehicleTrack) -> Dict[str, bool]:
        """Detect abnormalities based on spatio-temporal features."""
        abnormalities = {
            'over_speeding': False,
            'wrong_side': False,
            'wrong_turn': False,
            'sharp_turn': False
        }
        speeds = self.calculate_speed(track)
        if speeds and max(speeds) > self.speed_limit:
            abnormalities['over_speeding'] = True

        zone_seq = self.get_zone_sequence(track)
        seq_str = ''.join(map(str, zone_seq))
        for wrong_side_seq in self.violation_dict.get('wrong_side', []):
            if wrong_side_seq in seq_str:
                abnormalities['wrong_side'] = True
                break
        for wrong_turn_seq in self.violation_dict.get('wrong_turn', []):
            if wrong_turn_seq in seq_str:
                abnormalities['wrong_turn'] = True
                break

        angles = self.calculate_turn_angle(track)
        if angles and max(angles) > 90:
            abnormalities['sharp_turn'] = True

        return abnormalities

print("TrafficAnalyzer class defined.")

In [ ]:
def assign_zone_id(x: float, y: float) -> int:
    """Assign zone based on coordinates (customize for your road layout)."""
    if x < 250:
        return 1  # Example: North zone
    elif 250 <= x < 300:
        return 2  # Example: Middle zone
    else:
        return 3  # Example: South zone

def convert_df_to_track(df: pd.DataFrame) -> VehicleTrack:
    """Convert a DataFrame trajectory to a VehicleTrack."""
    track = []
    for _, row in df.iterrows():
        x = row['left']
        y = row['top']
        frame_id = int(row['frameNo'])
        zone_id = assign_zone_id(x, y)
        track.append(TrackPoint(frame_id=frame_id, x=x, y=y, zone_id=zone_id))
    return track

print("Helper functions defined.")

In [ ]:
def load_trajectories(video_folder: str) -> tuple[List[pd.DataFrame], List[int]]:
    """
    Load trajectory data from CSV files in a video folder.

    Parameters:
    - video_folder (str): Path to the video folder (e.g., '/content/sample_data/processed/10')

    Returns:
    - trajectories (list): List of pandas DataFrames, each containing a trajectory
    - labels (list): List of labels (0 for normal, 1 for abnormal)
    """
    normal_path = os.path.join(video_folder, 'normal')
    abnormal_path = os.path.join(video_folder, 'abnormal')

    trajectories = []
    labels = []

    # Load normal trajectories
    if os.path.exists(normal_path):
        normal_files = [os.path.join(normal_path, f) for f in os.listdir(normal_path) if f.endswith('.csv')]
        for file in normal_files:
            df = pd.read_csv(file)
            df = df[['frameNo', 'left', 'top', 'w', 'h']]
            df = df.sort_values(by='frameNo')
            trajectories.append(df)
            labels.append(0)

    # Load abnormal trajectories
    if os.path.exists(abnormal_path):
        abnormal_files = [os.path.join(abnormal_path, f) for f in os.listdir(abnormal_path) if f.endswith('.csv')]
        for file in abnormal_files:
            df = pd.read_csv(file)
            df = df[['frameNo', 'left', 'top', 'w', 'h']]
            df = df.sort_values(by='frameNo')
            trajectories.append(df)
            labels.append(1)

    return trajectories, labels

print("Trajectory loading function defined.")

In [ ]:
def extract_features_for_all_trajectories(trajectories: List[pd.DataFrame], labels: List[int],
                                         analyzer: TrafficAnalyzer) -> pd.DataFrame:
    """
    Extract features and detect abnormalities for all trajectories.

    Parameters:
    - trajectories: List of DataFrames with trajectory data
    - labels: List of labels (0=normal, 1=abnormal)
    - analyzer: TrafficAnalyzer instance

    Returns:
    - results_df: DataFrame with features and abnormalities
    """
    results = []
    for idx, (traj_df, label) in enumerate(zip(trajectories, labels)):
        track = convert_df_to_track(traj_df)
        abnormalities = analyzer.detect_abnormalities(track)
        speeds = analyzer.calculate_speed(track)
        angles = analyzer.calculate_turn_angle(track)
        zone_seq = analyzer.get_zone_sequence(track)

        result = {
            'track_id': idx + 1,
            'label': label,
            'over_speeding': abnormalities['over_speeding'],
            'wrong_side': abnormalities['wrong_side'],
            'wrong_turn': abnormalities['wrong_turn'],
            'sharp_turn': abnormalities['sharp_turn'],
            'max_speed': max(speeds) if speeds else 0.0,
            'avg_speed': np.mean(speeds) if speeds else 0.0,
            'max_turn_angle': max(angles) if angles else 0.0,
            'avg_turn_angle': np.mean(angles) if angles else 0.0,
            'zone_sequence': ','.join(map(str, zone_seq)),
            'num_points': len(track)
        }
        results.append(result)

    return pd.DataFrame(results)

print("Feature extraction function defined.")

In [ ]:
# Define base folder and video subfolders
base_folder = '/content/sample_data/processed'  # Adjust this path
video_folders = [
    os.path.join(base_folder, '10'),
    os.path.join(base_folder, '11'),
    os.path.join(base_folder, '12')
]

# Load all trajectories
all_trajectories = []
all_labels = []
for folder in video_folders:
    if os.path.exists(folder):
        traj, lbl = load_trajectories(folder)
        all_trajectories.extend(traj)
        all_labels.extend(lbl)
    else:
        print(f"Warning: Folder {folder} does not exist.")

print(f"Loaded {len(all_trajectories)} trajectories.")

# Initialize TrafficAnalyzer
zone_definitions = {
    1: {'type': 'entry', 'direction': 'N'},
    2: {'type': 'middle', 'direction': 'center'},
    3: {'type': 'exit', 'direction': 'S'}
}
violation_dict = {
    'wrong_side': ['31'],
    'wrong_turn': ['23']
}
analyzer = TrafficAnalyzer(
    fps=30.0,
    speed_limit=40.0,
    zone_definitions=zone_definitions,
    violation_dict=violation_dict,
    gsd=0.0396 * (50 ** 0.9478)
)

# Extract features
results_df = extract_features_for_all_trajectories(all_trajectories, all_labels, analyzer)

# Display results
print("\nSummary of Results:")
display(results_df[['track_id', 'label', 'over_speeding', 'wrong_side', 'wrong_turn', 'sharp_turn',
                   'max_speed', 'max_turn_angle']])

In [ ]:
# Save results to CSV
results_df.to_csv('/content/sample_data/trajectory_features.csv', index=False)
print("Features saved to '/content/sample_data/trajectory_features.csv'")

In [ ]:
# Load the CSV file
features_df = pd.read_csv('/content/sample_data/trajectory_features.csv')  # Replace with your file path if different

# Display the first few rows to inspect the data
print("Original Data Sample:")
display(features_df.head())

# Identify and drop noisy columns
numeric_cols = features_df.select_dtypes(include=[np.number]).columns
constant_cols = [col for col in numeric_cols if features_df[col].nunique() <= 1 or features_df[col].sum() == 0]
noisy_cols = ['max_speed', 'avg_speed', 'max_turn_angle', 'avg_turn_angle']  # Based on your data
cols_to_drop = list(set(constant_cols + noisy_cols))

# Drop noisy columns
cleaned_df = features_df.drop(columns=cols_to_drop)

# Convert boolean columns to integer (0 and 1) for analysis
bool_cols = ['over_speeding', 'wrong_side', 'wrong_turn', 'sharp_turn']
for col in bool_cols:
    cleaned_df[col] = cleaned_df[col].astype(int)

# Separate features (X) and labels (y)
X = cleaned_df.drop(columns=['track_id', 'label'])  # Features
y = cleaned_df['label']  # Labels

# Separate numeric and categorical features
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = ['zone_sequence']

print("\nCleaned Data Sample (after dropping noisy columns):")
display(cleaned_df.head())
print(f"Numeric Features: {numeric_features}")
print(f"Categorical Features: {categorical_features}")
print("Data cleaned and prepared successfully.")

In [ ]:
# Calculate mean and standard deviation for numeric features only
normal_group = cleaned_df[cleaned_df['label'] == 0]
abnormal_group = cleaned_df[cleaned_df['label'] == 1]

mean_normal = normal_group[numeric_features].mean()
std_normal = normal_group[numeric_features].std()
mean_abnormal = abnormal_group[numeric_features].mean()
std_abnormal = abnormal_group[numeric_features].std()

stats_df = pd.DataFrame({
    'Mean_Normal': mean_normal,
    'Std_Normal': std_normal,
    'Mean_Abnormal': mean_abnormal,
    'Std_Abnormal': std_abnormal
})

print("\nStatistical Summary for Numeric Features by Class (Mean and Std Dev):")
display(stats_df)

# Compute point-biserial correlation between each numeric feature and the label
correlations = {}
for col in numeric_features:
    corr, _ = pointbiserialr(y, X[col])
    correlations[col] = corr

# Convert to DataFrame for display
corr_df = pd.DataFrame(list(correlations.items()), columns=['Feature', 'Correlation'])
corr_df = corr_df.sort_values(by='Correlation', key=abs, ascending=False)

print("\nPoint-Biserial Correlation with Label (Numeric Features):")
display(corr_df)

# Visualize correlation values for numeric features
plt.figure(figsize=(10, 6))
plt.bar(corr_df['Feature'], corr_df['Correlation'], color='skyblue')
plt.axhline(0, color='gray', linewidth=0.5)
plt.xlabel('Feature')
plt.ylabel('Correlation with Label (0=Normal, 1=Abnormal)')
plt.title('Relevance of Numeric Features to Normal vs Abnormal')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Analyze zone_sequence (categorical) separately using frequency distribution
zone_counts = cleaned_df.groupby(['label', 'zone_sequence']).size().unstack(fill_value=0)
print("\nZone Sequence Distribution by Label:")
display(zone_counts)

# Calculate a chi-squared test to assess association between zone_sequence and label
from scipy.stats import chi2_contingency
chi2, p, _, _ = chi2_contingency(zone_counts)
print(f"\nChi-Squared Test for zone_sequence vs Label:")
print(f"Chi2 Statistic: {chi2:.2f}, p-value: {p:.4f}")
print("A low p-value (<0.05) indicates a significant association between zone_sequence and label.")

In [ ]:
# Summary of findings
print("\nRelevance Assessment:")
print("- Numeric features with |Correlation| > 0.3 are strong indicators for classification.")
print("- Significant differences in means between normal and abnormal classes suggest relevance.")
print("- For zone_sequence, a low p-value (<0.05) in the chi-squared test indicates relevance.")

# Highlight strongly relevant numeric features
relevant_numeric_features = corr_df[abs(corr_df['Correlation']) > 0.3]['Feature'].tolist()
if relevant_numeric_features:
    print(f"Strongly relevant numeric features: {relevant_numeric_features}")
else:
    print("No numeric features with strong correlation (|corr| > 0.3).")

# Highlight zone_sequence relevance
if p < 0.05:
    print("zone_sequence appears relevant based on the chi-squared test (p < 0.05).")
else:
    print("zone_sequence may not be strongly relevant (p >= 0.05). Consider encoding or deriving new features from it.")

# Recommendations
print("\nRecommendations:")
if relevant_numeric_features or p < 0.05:
    print("- Some features appear relevant for distinguishing normal vs abnormal tracks.")
    print("- Proceed with these features for classification or further analysis.")
else:
    print("- Features lack strong relevance. Consider the following:")
    print("  - Verify data quality: Ensure trajectories capture meaningful motion (speed, turns, etc.).")
    print("  - Feature engineering: Derive new features (e.g., zone transitions, sequence length).")
    print("  - Expand dataset: Include more diverse trajectories to increase feature variation.")